In [3]:
###Testing
import SimFunctions
import SimRNG 
import SimClasses
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as st

def sim(probArrH, probHighALC, probLowALC,MeanPTHX1,MeanPTLX1):  
    ZSimRNG = SimRNG.InitializeRNSeed()

    QueueH = SimClasses.FIFOQueue()
    QueueL = SimClasses.FIFOQueue()
    WaitH = SimClasses.DTStat()
    WaitL = SimClasses.DTStat()
    AbandonH = SimClasses.DTStat()
    AbandonL = SimClasses.DTStat()
    Server = SimClasses.Resource()
    Calendar = SimClasses.EventCalendar()

    TheCTStats = []
    TheDTStats = []
    TheQueues = []
    TheResources = []

    TheDTStats.append(WaitH)
    TheDTStats.append(WaitL)
    TheDTStats.append(AbandonH)
    TheDTStats.append(AbandonL)
    TheQueues.append(QueueH)
    TheQueues.append(QueueL)
    TheResources.append(Server)

    numServer = 4
    Server.SetUnits(numServer)
    # Define the arrival rate and interarrival time
    arrRate = 15 # lambda
    probArrH = probArrH #qH
    probArrL = 1.0 - probArrH #qL
    MeanTBAH = 1.0/(arrRate*probArrH)
    MeanTBAL = 1.0/(arrRate*probArrL)

    # Define the mean service time for class H and L
    MeanSTH = 0.25
    MeanSTL = 0.25
    Phases = 1

    # Mean abandonment time for customers
    MeanPTHX1 = MeanPTHX1# Ahigh | Xhigh = 1
    MeanPTHX0 = MeanPTHX1 # Ahigh | Xhigh = 0
    MeanPTLX1 = MeanPTLX1# Alow | Xhigh = 1
    MeanPTLX0 = MeanPTLX1 # Alow | Xhigh = 0

    # ALC Risk
    probHighALC = probHighALC
    probLowALC = probLowALC

    # RunLength = 14400.0
    # WarmUp = 4320.0
    RunLength = 5500.0
    WarmUp = 500.0

    Cost = []
    
    # Functions
    def ArrivalHigh(Customer):  
        if Customer.X == 1:
            SimFunctions.SchedulePlus(Calendar,"AbandonHigh",SimRNG.Expon(MeanPTHX1, 1),Customer)
        elif Customer.X == 0:
            SimFunctions.SchedulePlus(Calendar,"AbandonHigh",SimRNG.Expon(MeanPTHX0, 1),Customer)

        if Server.Busy < numServer:
            WaitH.Record(0)
            Server.Seize(1)
            SimFunctions.Schedule(Calendar,"EndOfServiceHigh",SimRNG.Erlang(Phases,MeanSTH,1))
        else:
            QueueH.Add(Customer) 

        # Schedule the next arrival with SchedulePlus
        NewCustomer = SimClasses.Entity()
        NewCustomer.ClassNum = 0
        NewCustomer.id = Customer.id + 1
        NewCustomer.setRealization(probHighALC)
        SimFunctions.SchedulePlus(Calendar,"ArrivalHigh",SimRNG.Expon(MeanTBAH, 1),NewCustomer)

    def ArrivalLow(Customer):
        if Customer.X == 1:
            SimFunctions.SchedulePlus(Calendar,"AbandonLow",SimRNG.Expon(MeanPTLX1, 1),Customer)
        elif Customer.X == 0:
            SimFunctions.SchedulePlus(Calendar,"AbandonLow",SimRNG.Expon(MeanPTLX0, 1),Customer)

        if Server.Busy < numServer:
            WaitL.Record(0)
            Server.Seize(1)
            SimFunctions.Schedule(Calendar,"EndOfServiceLow",SimRNG.Erlang(Phases,MeanSTL,1))
        else:
            QueueL.Add(Customer) 

        # Schedule the next arrival with SchedulePlus
        NewCustomer = SimClasses.Entity()
        NewCustomer.ClassNum = 1
        NewCustomer.id = Customer.id + 1
        NewCustomer.setRealization(probLowALC)
        SimFunctions.SchedulePlus(Calendar,"ArrivalLow",SimRNG.Expon(MeanTBAL, 1),NewCustomer)

    def AbandonHigh(Customer):
        # When it's time for abandonment,
        # Check if the customer is still in queue, if yes, remove from queue
        if QueueH.isInQueue(Customer):
            if Customer.X == 1:
                AbandonH.Record(1)
            else:
                AbandonH.Record(0)
            QueueH.RemoveCustomer(Customer)

    def AbandonLow(Customer):
        # When it's time for abandonment,
        # Check if the customer is still in queue, if yes, remove from queue
        if QueueL.isInQueue(Customer):
            if Customer.X == 1:
                AbandonL.Record(1)
            else:
                AbandonL.Record(0)
            QueueL.RemoveCustomer(Customer)

    def EndOfServiceHigh():
        AbandonH.Record(0)
        if QueueH.NumQueue() > 0:
            DepartingCustomer = QueueH.Remove()
            WaitH.Record(SimClasses.Clock - DepartingCustomer.CreateTime)
            SimFunctions.Schedule(Calendar,"EndOfServiceHigh",SimRNG.Erlang(Phases,MeanSTH,1))
        elif QueueL.NumQueue() > 0:
            DepartingCustomer = QueueL.Remove()
            WaitL.Record(SimClasses.Clock - DepartingCustomer.CreateTime)
            SimFunctions.Schedule(Calendar,"EndOfServiceLow",SimRNG.Erlang(Phases,MeanSTL,1))
        else:
            Server.Free(1)

    def EndOfServiceLow():
        AbandonL.Record(0)
        if QueueH.NumQueue() > 0:
            DepartingCustomer = QueueH.Remove()
            WaitH.Record(SimClasses.Clock - DepartingCustomer.CreateTime)
            SimFunctions.Schedule(Calendar,"EndOfServiceHigh",SimRNG.Erlang(Phases,MeanSTH,1))
        elif QueueL.NumQueue() > 0:
            DepartingCustomer = QueueL.Remove()
            WaitL.Record(SimClasses.Clock - DepartingCustomer.CreateTime)
            SimFunctions.Schedule(Calendar,"EndOfServiceLow",SimRNG.Erlang(Phases,MeanSTL,1))
        else:
            Server.Free(1)

    def CI_95(data):
        a = np.array(data, dtype=object)
        n = len(a)
        m = np.mean(a)
        sd = np.std(a,ddof=1)
        hw = 1.96*sd / np.sqrt(n)
#         return m, hw, m-hw,m+hw
#         return m, hw, [m-hw,m+hw]
        return m-hw, m+hw
    
    for reps in range(0,30,1):
        # Initialize the system
        SimFunctions.SimFunctionsInit(Calendar,TheQueues,TheCTStats,TheDTStats,TheResources)

        # Schedule arrival for high ALC
        FirstCustomer = SimClasses.Entity()
        FirstCustomer.ClassNum = 0
        FirstCustomer.setRealization(probHighALC)
        SimFunctions.SchedulePlus(Calendar,"ArrivalHigh",SimRNG.Expon(MeanTBAH, 1),FirstCustomer)

        # Schedule arrival for Low ALC
        SecCustomer = SimClasses.Entity()
        SecCustomer.ClassNum = 1
        SecCustomer.setRealization(probLowALC)
        SimFunctions.SchedulePlus(Calendar,"ArrivalLow",SimRNG.Expon(MeanTBAL, 1),SecCustomer)
        SimFunctions.Schedule(Calendar,"EndSimulation",RunLength)
        SimFunctions.Schedule(Calendar,"ClearIt",WarmUp)

        NextEvent = Calendar.Remove()
        SimClasses.Clock = NextEvent.EventTime
        if NextEvent.EventType == "ArrivalHigh":
            ArrivalHigh(NextEvent.WhichObject)
        elif NextEvent.EventType == "ArrivalLow":
            ArrivalLow(NextEvent.WhichObject)

        while NextEvent.EventType != "EndSimulation":
            NextEvent = Calendar.Remove()
            SimClasses.Clock = NextEvent.EventTime
            if NextEvent.EventType == "ArrivalHigh":
                ArrivalHigh(NextEvent.WhichObject)
            elif NextEvent.EventType == "ArrivalLow":
                ArrivalLow(NextEvent.WhichObject)
            elif NextEvent.EventType == "AbandonHigh":
                AbandonHigh(NextEvent.WhichObject)
            elif NextEvent.EventType == "AbandonLow":
                AbandonLow(NextEvent.WhichObject)
            elif NextEvent.EventType == "EndOfServiceHigh":
                EndOfServiceHigh() 
            elif NextEvent.EventType == "EndOfServiceLow":
                EndOfServiceLow() 
            elif NextEvent.EventType == "ClearIt":
                SimFunctions.ClearStats(TheCTStats,TheDTStats)
#         AllAbandonH.append(AbandonH.Mean())
#         AllAbandonL.append(AbandonL.Mean())
        Cost.append((AbandonH.Sum+AbandonL.Sum)/(AbandonH.NumberOfObservations+AbandonL.NumberOfObservations))
#     return np.mean(AllAbandonH), np.mean(AllAbandonL), np.mean(Cost), CI_95(Cost)
    return Cost

In [4]:
# probArrH, probHighALC, probLowALC,MeanPTHX1,MeanPTLX1
C1= sim(0.3,0.4,0.3,1,1)
C2 = sim(0.7,0.3,0.4,1,1)

data = np.array(C1[0:-4]) - np.array(C2[0:-4])
mean = np.mean(data)
(lb, ub) = st.norm.interval(alpha=0.95, loc=mean, scale=st.sem(data))
data = np.append(data, [mean, lb, ub, mean-lb], axis = 0)

# convert the output to a dataframe and write to a .csv file
output = {"P1Cost" : C1, "P2Cost": C2, "E[c1-c2]" : data}
output = pd.DataFrame(output)
output.to_csv("0.4_0.3.csv", sep=",")

In [5]:
# probArrH, probHighALC, probLowALC,MeanPTHX1,MeanPTLX1
C1= sim(0.3,0.6,0.4,1,1)
C2 = sim(0.7,0.4,0.6,1,1)

data = np.array(C1[0:-4]) - np.array(C2[0:-4])
mean = np.mean(data)
(lb, ub) = st.norm.interval(alpha=0.95, loc=mean, scale=st.sem(data))
data = np.append(data, [mean, lb, ub, mean-lb], axis = 0)

# convert the output to a dataframe and write to a .csv file
output = {"P1Cost" : C1, "P2Cost": C2, "E[c1-c2]" : data}
output = pd.DataFrame(output)
output.to_csv("0.6_0.4.csv", sep=",")

In [6]:
# probArrH, probHighALC, probLowALC,MeanPTHX1,MeanPTLX1
C1= sim(0.3,0.7,0.3,1,1)
C2 = sim(0.7,0.3,0.7,1,1)

data = np.array(C1[0:-4]) - np.array(C2[0:-4])
mean = np.mean(data)
(lb, ub) = st.norm.interval(alpha=0.95, loc=mean, scale=st.sem(data))
data = np.append(data, [mean, lb, ub, mean-lb], axis = 0)

# convert the output to a dataframe and write to a .csv file
output = {"P1Cost" : C1, "P2Cost": C2, "E[c1-c2]" : data}
output = pd.DataFrame(output)
output.to_csv("0.7_0.3.csv", sep=",")

In [7]:
# probArrH, probHighALC, probLowALC,MeanPTHX1,MeanPTLX1
C1= sim(0.3,0.8,0.2,1,1)
C2 = sim(0.7,0.2,0.8,1,1)

data = np.array(C1[0:-4]) - np.array(C2[0:-4])
mean = np.mean(data)
(lb, ub) = st.norm.interval(alpha=0.95, loc=mean, scale=st.sem(data))
data = np.append(data, [mean, lb, ub, mean-lb], axis = 0)

# convert the output to a dataframe and write to a .csv file
output = {"P1Cost" : C1, "P2Cost": C2, "E[c1-c2]" : data}
output = pd.DataFrame(output)
output.to_csv("0.8_0.2.csv", sep=",")

In [8]:
# probArrH, probHighALC, probLowALC,MeanPTHX1,MeanPTLX1
C1= sim(0.3,0.9,0.1,1,1)
C2 = sim(0.7,0.1,0.9,1,1)

data = np.array(C1[0:-4]) - np.array(C2[0:-4])
mean = np.mean(data)
(lb, ub) = st.norm.interval(alpha=0.95, loc=mean, scale=st.sem(data))
data = np.append(data, [mean, lb, ub, mean-lb], axis = 0)

# convert the output to a dataframe and write to a .csv file
output = {"P1Cost" : C1, "P2Cost": C2, "E[c1-c2]" : data}
output = pd.DataFrame(output)
output.to_csv("0.9_0.1.csv", sep=",")